In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CrossAttention(nn.Module):
    def __init__(self, query_dim, context_dim):
        super(CrossAttention, self).__init__()
        self.query_dim = query_dim
        self.context_dim = context_dim

        self.linear_q = nn.Linear(query_dim, query_dim)
        self.linear_c = nn.Linear(context_dim, query_dim)

    def forward(self, query, context):
        # Query和Context的维度分别为 [batch_size, query_len, query_dim] 和 [batch_size, context_len, context_dim]
        # 首先将Query和Context分别通过线性变换
        query_proj = self.linear_q(query)  # [batch_size, query_len, query_dim]
        context_proj = self.linear_c(context)  # [batch_size, context_len, query_dim]

        # 计算注意力权重
        attention_weights = torch.bmm(query_proj, context_proj.transpose(1, 2))  # [batch_size, query_len, context_len]
        attention_weights = F.softmax(attention_weights, dim=-1)

        # 对Context序列进行加权求和
        attended_context = torch.bmm(attention_weights, context)  # [batch_size, query_len, context_dim]

        return attended_context, attention_weights



In [3]:
tensor1 = torch.randn(10, 3, 4)  
tensor2 = torch.randn(4, 5)  
torch.matmul(tensor1, tensor2).size()  

torch.Size([10, 3, 5])

In [14]:
CrossAttention(4,5).forward(tensor1, tensor2)

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CrossAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(CrossAttention, self).__init__()
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scaling = self.head_dim ** -0.5
        
        self.q_linear = nn.Linear(embed_dim, embed_dim)
        self.k_linear = nn.Linear(embed_dim, embed_dim)
        self.v_linear = nn.Linear(embed_dim, embed_dim)
        self.out_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, query, key, value):
        # Shape: (batch_size, seq_len, embed_dim)
        batch_size = query.size(0)

        # Linear projections
        q = self.q_linear(query)
        k = self.k_linear(key)
        v = self.v_linear(value)

        # Reshape to (batch_size, num_heads, seq_len, head_dim)
        q = q.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        k = k.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        v = v.view(batch_size, -1, self.num_heads, self.head_dim).transpose(1, 2)
        
        # Attention scores
        attn_scores = torch.einsum('bhqd,bhkd->bhqk', q, k) * self.scaling
        attn_weights = F.softmax(attn_scores, dim=-1)

        # Weighted sum of values
        attn_output = torch.einsum('bhqk,bhvd->bhqd', attn_weights, v)

        # Reshape back to (batch_size, seq_len, embed_dim)
        attn_output = attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads * self.head_dim)
        
        return self.out_linear(attn_output)